In [2]:
import docx2json
import json
import psycopg2
import getpass
import pandas as pd
from os import listdir, getcwd

In [6]:
conn = psycopg2.connect("host=35.192.163.239 port=5432 dbname=testdb user=%s password=%s" % (input("User: "), getpass.getpass("Password: ")))
cur = conn.cursor()

In [34]:
cur = conn.cursor()
cur.execute("""EXPLAIN (ANALYZE TRUE, TIMING FALSE) SELECT ID FROM TEXTARRTEST
            WHERE to_tsvector('portuguese', TEXTO) @@ phraseto_tsquery('portuguese', 'nasci em londrina');""")
print(*cur.fetchall(), sep='\n')

('Seq Scan on textarrtest  (cost=0.00..11.73 rows=1 width=4) (actual rows=1 loops=1)',)
("  Filter: (to_tsvector('portuguese'::regconfig, texto) @@ '''nasc'' <2> ''londrin'''::tsquery)",)
('  Rows Removed by Filter: 17',)
('Planning time: 0.133 ms',)
('Execution time: 159.029 ms',)


In [18]:
conn.rollback()

In [ ]:
# Creating textsearch column and index
# cur = conn.cursor()
# cur.execute("""ALTER TABLE TEXTARRTEST ADD COLUMN textsearch_index_col tsvector;""")
# cur.execute("""UPDATE textarrtest SET textsearch_index_col = to_tsvector('portuguese', coalesce(texto, ''));""")
# cur.execute("""CREATE INDEX idx_textarrtest_textsearch ON textarrtest USING GIN (textsearch_index_col);""")
# conn.commit()

In [36]:
# Forcing index usage for testing purposes
cur = conn.cursor()
cur.execute("""SET enable_seqscan = off;""")
cur.execute("""EXPLAIN (ANALYZE TRUE, TIMING FALSE) SELECT ID FROM TEXTARRTEST
            WHERE textsearch_index_col @@ phraseto_tsquery('portuguese', 'nasci em londrina');""")
print(*cur.fetchall(), sep='\n')
cur.execute("""SET enable_seqscan = on;""")
conn.commit()

('Bitmap Heap Scan on textarrtest  (cost=20.08..27.54 rows=10 width=4) (actual rows=1 loops=1)',)
("  Recheck Cond: (textsearch_index_col @@ '''nasc'' <2> ''londrin'''::tsquery)",)
('  Rows Removed by Index Recheck: 9',)
('  Heap Blocks: exact=3',)
('  ->  Bitmap Index Scan on idx_textarrtest_textsearch  (cost=0.00..20.08 rows=10 width=0) (actual rows=10 loops=1)',)
("        Index Cond: (textsearch_index_col @@ '''nasc'' <2> ''londrin'''::tsquery)",)
('Planning time: 0.208 ms',)
('Execution time: 0.260 ms',)


In [48]:
conn.rollback()

In [50]:
# Creating trigger for textsearch column
cur = conn.cursor()
cur.execute("""CREATE TRIGGER trig_textarrtest_textsearchupdate BEFORE INSERT OR UPDATE
            ON textarrtest FOR EACH ROW EXECUTE PROCEDURE
            tsvector_update_trigger(textsearch_index_col, 'pg_catalog.portuguese', texto);""")
conn.commit()

In [55]:
cur = conn.cursor()
cur.execute("""SELECT * FROM ts_stat('SELECT textsearch_index_col FROM textarrtest')
            ORDER BY nentry DESC, ndoc DESC, word ASC
            LIMIT 10;""")
print(*cur.fetchall(), sep='\n')

('é', 18, 2631)
('assim', 18, 1681)
('gent', 18, 1630)
('pra', 18, 1409)
('entã', 18, 1174)
('porqu', 18, 985)
('aí', 16, 940)
('trein', 18, 738)
('faz', 18, 690)
('fal', 18, 678)
